In [1]:
import ee
import geemap
import matplotlib.pyplot as plt
import numpy as np
import IPython.display as disp


ee.Authenticate()
ee.Initialize(project='norse-fragment-435517-g1')

In [2]:
#PREPROCESSING
def mask_edge(image):
    edge = image.lt(-30.0)
    masked_image = image.mask().And(edge.Not())
    return image.updateMask(masked_image)

def apply_lee_filter(image):
    kernel_size = 3
    nat = ee.Image.constant(10).pow(image.divide(10))
    kernel = ee.Kernel.square(kernel_size, 'pixels')
    mean = nat.reduceNeighborhood(reducer=ee.Reducer.mean(), kernel=kernel)
    variance = nat.reduceNeighborhood(reducer=ee.Reducer.variance(), kernel=kernel)

    weight = variance.divide(variance.add(ee.Image.constant(0.01)))

    result = ee.Image.constant(1).subtract(weight).multiply(mean).add(weight.multiply(nat))

    return ee.Image(result).log10().multiply(10)

def iterative_lee_filter(image, iterations):
    for _ in range(iterations):
        image = apply_lee_filter(image)
    return image


In [3]:
#CLASSIFICATION
def apply_spatial_smoothing(image, kernel_size, smoothing_factor):
    kernel = ee.Kernel.square(kernel_size, 'pixels')
    smoothed = image.convolve(kernel)
    return ee.Image(1).subtract(ee.Image.constant(smoothing_factor)).multiply(image).add(ee.Image.constant(smoothing_factor).multiply(smoothed))

def connected_pixel_aggregation(clustered_image, min_size=15):
    connected = clustered_image.connectedPixelCount()
    large_clusters = connected.gte(min_size)
    return clustered_image.updateMask(large_clusters)

def majority_filter(image, kernel_size, smoothing_factor):
    kernel = ee.Kernel.square(kernel_size, 'pixels')
    mode = image.reduceNeighborhood(reducer=ee.Reducer.mode(), kernel=kernel)
    return ee.Image(1).subtract(ee.Image.constant(smoothing_factor)).multiply(image).add(ee.Image.constant(smoothing_factor).multiply(mode))

def homogeneous_kmeans_clustering(image, region, n_clusters, scale=30, num_pixels=100000, seed=42, smoothing_factor=0.5, kernel_size=2):
    smoothed_image = apply_spatial_smoothing(image, kernel_size, smoothing_factor)

    training = smoothed_image.sample(
        region=region,
        scale=scale,
        numPixels=num_pixels,
        seed=seed
    )

    clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training)
    clustered = smoothed_image.cluster(clusterer)

    aggregated = connected_pixel_aggregation(clustered)

    result = majority_filter(aggregated, kernel_size, smoothing_factor)

    return result

In [6]:
s1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterDate('2021-01-01', '2021-12-31') \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')) \
    .select('VV') \
    .mean()

roi = ee.Geometry.Polygon([[77.6320454445987,13.220795669092979],
            [77.6320454445987,13.127228632479046],
            [77.76267144567248,13.127228632479046],
            [77.76267144567248,13.220795669092979],
            [77.6320454445987,13.220795669092979]])

lee_filtered = iterative_lee_filter(s1, iterations=3)

# Parameters (adjust as needed)
smoothing_factor = 0
kernel_size = 0

clustered = homogeneous_kmeans_clustering(lee_filtered, roi, n_clusters=5, smoothing_factor=smoothing_factor, kernel_size=kernel_size)

masked = mask_edge(clustered)

Map = geemap.Map()
Map.addLayer(masked, {'min': 0, 'max': 5, 'palette': ['blue', 'green', 'red','purple','yellow']}, 'Clustered Image')
Map.addLayer(s1, {'min': -25, 'max': 0}, 'SAR Image')
Map.centerObject(roi, 12)
Map

Map(center=[13.174017431165826, 77.69735844513708], controls=(WidgetControl(options=['position', 'transparent_…